# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [16]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [18]:
df = pd.read_csv('/content/dataset.csv')
places = df['SystemCodeNumber'].unique()
places

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

array(['BHMBCCMKT01', 'BHMBCCTHL01', 'BHMEURBRD01', 'BHMMBMMBX01',
       'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a',
       'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202',
       'Others-CCCPS8', 'Others-CCCPS98', 'Shopping'], dtype=object)

In [19]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [20]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [21]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber : str
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location


In [22]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [23]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [24]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time
    .windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        SystemCodeNumber = pw.reducers.argmax(pw.this.t, pw.this.SystemCodeNumber),
        occupancy = pw.reducers.argmax(pw.this.t, pw.this.Occupancy),
        capacity = pw.reducers.argmax(pw.this.t, pw.this.Capacity),
        t = pw.reducers.argmax(pw.this.t, pw.this.t),
        day = pw.reducers.argmax(pw.this.t, pw.this.day),
    )
)

delta_window


<pathway.Table schema={'SystemCodeNumber': <class 'str'>, 'occupancy': <class 'int'>, 'capacity': <class 'int'>, 't': <class 'pathway.internals.datetime_types.DateTimeNaive'>, 'day': <class 'str'>} id_type=pathway.engine.Pointer[str]>

In [25]:
pw.io.csv.write(delta_window, '/content/output.csv')

In [26]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

In [27]:
df = pd.read_csv('/content/output.csv')
df['t'] = pd.to_datetime(df['t'])
df['price'] = 10
df

,SystemCodeNumber,occupancy,capacity,t,day,time,diff,price
0,BHMEURBRD01,325,470,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10
1,BHMNCPNST01,304,485,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10
2,Others-CCCPS98,936,3103,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10
3,BHMBCCTHL01,245,387,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10
4,Others-CCCPS105a,1089,2009,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10
...,...,...,...,...,...,...,...,...
2025,Others-CCCPS8,806,1322,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,10
2026,BHMNCPNST01,257,485,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,10
2027,Others-CCCPS135a,2533,3883,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,10
2028,BHMEURBRD01,373,470,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,10


In [28]:
for place in places:
   mask = df['SystemCodeNumber'] == place
   df_1 = df[mask]
   data = df_1.copy().reset_index(drop=True)


#Update prices row-by-row using previous row
   for i in range(1, len(data)):
    prev_price = data.loc[i - 1, 'price']
    prev_occ = data.loc[i-1, 'occupancy']
    prev_cap = data.loc[i-1, 'capacity']
    occ = data.loc[i, 'occupancy']
    cap = data.loc[i, 'capacity']
    if (occ/cap) < (prev_occ/prev_cap):
      alpha = -0.2
    else:
      alpha = 0.2

    data.loc[i, 'price'] = prev_price + alpha * (occ / cap)



   df.loc[mask, 'price'] = data['price'].values


df


/tmp/ipython-input-28-1936129943.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.049566724436742' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'price'] = prev_price + alpha * (occ / cap)
/tmp/ipython-input-28-1936129943.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.         10.04956672 10.01005199 10.04956672 10.09428076 10.1389948
 10.1847487  10.2305026  10.35701906 10.48353553 10.43743501 10.48353553
 10.4440208  10.48353553 10.52755633 10.57157712 10.53310225 10.57157712
 10.61663778 10.66169844 10.70814558 10.75459272 10.89081456 11.0270364
 10.97365685 11.0270364  10.98890815 11.0270364  11.07001733 11.11299827
 11.07487002 11.11299827 11.23084922 11.34870017 11.29670711 11.34870017
 11.40935875 11.47001733 11.54835355 11.62668977 11.56429809 

,SystemCodeNumber,occupancy,capacity,t,day,time,diff,price
0,BHMEURBRD01,325,470,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10.000000
1,BHMNCPNST01,304,485,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10.000000
2,Others-CCCPS98,936,3103,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10.000000
3,BHMBCCTHL01,245,387,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10.000000
4,Others-CCCPS105a,1089,2009,2016-10-04 16:31:00,2016-10-04T00:00:00,1751720091206,1,10.000000
...,...,...,...,...,...,...,...,...
2025,Others-CCCPS8,806,1322,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,18.339183
2026,BHMNCPNST01,257,485,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,19.776082
2027,Others-CCCPS135a,2533,3883,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,17.640690
2028,BHMEURBRD01,373,470,2016-12-19 16:30:00,2016-12-19T00:00:00,18446744073709551614,1,20.245957


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [29]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import curdoc
from bokeh.plotting import ColumnDataSource, figure, output_file, show


# print(df)

curdoc().theme = 'dark_minimal'


def plot_price_fluctuations(df, place):
   p = figure(title=place ,width=1000, height=400, x_axis_type="datetime")
   p.line(df.t, df.price)
   p.scatter(df.t, df.price, fill_color="red", size=5)
   show(p)


for place in places:
   plot_price_fluctuations(df[df['SystemCodeNumber'] == place], place)


In [ ]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure
from bokeh.layouts import column
import pandas as pd

# Example: df = your DataFrame with columns: ['SystemCodeNumber', 't', 'price']
# Make sure 't' is in datetime format:
# df['t'] = pd.to_datetime(df['t'])

# Define unique places
places = sorted(df['SystemCodeNumber'].unique())

# Create a default source with the first place's data
default_place = places[0]
initial_data = df[df['SystemCodeNumber'] == default_place]
source = ColumnDataSource(initial_data)

# Set theme
curdoc().theme = 'dark_minimal'

# Create the dropdown widget
dropdown = Select(title="Select Place", value=default_place, options=list(places))

# Create the figure
plot = figure(title=f"Price Fluctuations: {default_place}", width=1000, height=400, x_axis_type="datetime")
line = plot.line(x='t', y='price', source=source, line_width=2)
scatter = plot.scatter(x='t', y='price', source=source, fill_color="red", size=5)

# Callback function to update the plot
def update_plot(attr, old, new):
    selected_place = dropdown.value
    new_data = df[df['SystemCodeNumber'] == selected_place]
    source.data = ColumnDataSource(new_data).data
    plot.title.text = f"Price Fluctuations: {selected_place}"

# Link dropdown to callback
dropdown.on_change('value', update_plot)

# Layout and add to document
layout = column(dropdown, plot)
curdoc().add_root(layout)
